In [55]:
%pip install -q ccxt pandas numpy nbformat bokeh

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from __future__ import annotations
import time
from datetime import datetime, timezone, timedelta
from typing import List, Dict, Any, Optional, Tuple

import ccxt
import pandas as pd
import numpy as np

# ---------- Sessions / ORB config (UTC times)
DEFAULT_SESSIONS = [
    {"name": "asia",      "start_utc": "00:00", "end_utc": "08:00"},
    {"name": "europe",    "start_utc": "08:00", "end_utc": "13:00"},
    {"name": "us",        "start_utc": "13:00", "end_utc": "21:00"},
    {"name": "overnight", "start_utc": "21:00", "end_utc": "24:00"},
]

# ---------- Utilities

def to_millis(dt: datetime) -> int:
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return int(dt.timestamp() * 1000)

def ensure_usdt_symbols(symbols: List[str]) -> List[str]:
    return [s for s in symbols if s.endswith("/USDT")]

def interval_to_ccxt_tf(tf: str) -> str:
    return tf  # '1m','5m','15m','1h','4h','1d'

# ---------- Technicals

def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()

def rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff()
    up = np.where(delta > 0, delta, 0.0)
    down = np.where(delta < 0, -delta, 0.0)
    roll_up = pd.Series(up, index=series.index).ewm(alpha=1/period, adjust=False).mean()
    roll_down = pd.Series(down, index=series.index).ewm(alpha=1/period, adjust=False).mean()
    rs = roll_up / (roll_down.replace(0, np.nan))
    rsi_val = 100 - (100 / (1 + rs))
    return rsi_val.fillna(0.0)

def macd(series: pd.Series, fast=12, slow=26, signal=9) -> Tuple[pd.Series, pd.Series, pd.Series]:
    ema_fast = ema(series, fast)
    ema_slow = ema(series, slow)
    macd_line = ema_fast - ema_slow
    signal_line = ema(macd_line, signal)
    hist = macd_line - signal_line
    return macd_line, signal_line, hist

def rolling_percentiles(series: pd.Series, window: int = 50,
                        percentiles: Tuple[float, float, float] = (0.2, 0.5, 0.8)
                       ) -> Tuple[pd.Series, pd.Series, pd.Series]:
    p_low, p_mid, p_high = percentiles
    def _roll_quantile(s, q):
        return s.rolling(window, min_periods=max(2, int(window/5))).quantile(q)
    return (_roll_quantile(series, p_low),
            _roll_quantile(series, p_mid),
            _roll_quantile(series, p_high))

# ---------- Daily (UTC) ORB

def mark_orb_daily(df: pd.DataFrame) -> pd.DataFrame:
    """First candle of each UTC day as ORB; add L1/L2/mid."""
    if df.empty:
        return df
    out = df.copy()
    idx = pd.to_datetime(out.index, utc=True)
    out.index = idx
    dates = idx.normalize().date
    out["__date"] = dates
    first_idx = out.groupby("__date", sort=False).head(1).index
    out["is_orb"] = out.index.isin(first_idx)
    orb_high = out.loc[first_idx, "high"].rename("orb_high")
    orb_low  = out.loc[first_idx, "low"].rename("orb_low")
    per_day = pd.DataFrame({"orb_high": orb_high, "orb_low": orb_low})
    per_day["orb_mid"] = (per_day["orb_high"] + per_day["orb_low"]) / 2.0
    rng = (per_day["orb_high"] - per_day["orb_low"])
    per_day["L1_bull"] = per_day["orb_high"] + 0.5 * rng
    per_day["L2_bull"] = per_day["L1_bull"] + 0.5 * (per_day["L1_bull"] - per_day["orb_high"])
    per_day["L1_bear"] = per_day["orb_low"] - 0.5 * rng
    per_day["L2_bear"] = per_day["L1_bear"] - 0.5 * (per_day["orb_low"] - per_day["L1_bear"])
    per_day["__date"] = per_day.index.tz_convert("UTC").normalize().date
    out = out.merge(
        per_day.reset_index()[["__date","orb_high","orb_low","orb_mid","L1_bull","L2_bull","L1_bear","L2_bear"]],
        on="__date", how="left"
    ).set_index(idx)
    out = out.drop(columns=["__date"])
    return out

# ---------- Per-session ORBs (UTC windows)

def _hhmm_to_offset(hhmm: str) -> pd.Timedelta:
    h, m = map(int, hhmm.split(":"))
    return pd.Timedelta(hours=h, minutes=m)

def add_sessions_and_orbs(df: pd.DataFrame, sessions: List[Dict[str, str]]) -> pd.DataFrame:
    """
    Adds, per session:
      - is_orb_<name>
      - orb_high_<name>, orb_low_<name>, orb_mid_<name>
      - L1_bull_<name>, L2_bull_<name>, L1_bear_<name>, L2_bear_<name>
      - session_id_<name> (YYYY-MM-DD_<name>)
    """
    if df.empty:
        return df

    out = df.copy()
    idx_utc = pd.to_datetime(out.index, utc=True)
    out.index = idx_utc

    # midnight-UTC anchor per row
    base = idx_utc.normalize()

    for s in sessions:
        nm = s["name"]
        start_off = _hhmm_to_offset(s["start_utc"])
        end_off   = _hhmm_to_offset(s["end_utc"])

        start_ts = base + start_off
        end_ts   = base + end_off

        # handle wrap past midnight (e.g., 21:00 → 24:00 or start > end)
        wrap = end_ts <= start_ts
        end_ts = end_ts.where(~wrap, end_ts + pd.Timedelta(days=1))

        in_session = (idx_utc >= start_ts) & (idx_utc < end_ts)

        # stable session id per row → based on the *start day* of the window
        sess_id = (base + start_off).strftime("%Y-%m-%d") + "_" + nm
        sess_id_arr = np.where(in_session, np.asarray(sess_id), pd.NA)
        out[f"session_id_{nm}"] = sess_id_arr

        # first candle (ORB) within each session
        if in_session.any():
            first_idx = out.loc[in_session].groupby(f"session_id_{nm}", dropna=True).head(1).index
        else:
            first_idx = out.index[[]]
        out[f"is_orb_{nm}"] = False
        out.loc[first_idx, f"is_orb_{nm}"] = True

        # broadcast ORB high/low/mid and L1/L2 across rows that are in the session
        if in_session.any():
            g = out.loc[in_session].groupby(f"session_id_{nm}", dropna=True)
            orb_high = g["high"].transform("first")
            orb_low  = g["low"].transform("first")
            mid = (orb_high + orb_low) / 2.0
            rng = (orb_high - orb_low)
            L1_bull = orb_high + 0.5 * rng
            L2_bull = L1_bull + 0.5 * (L1_bull - orb_high)
            L1_bear = orb_low  - 0.5 * rng
            L2_bear = L1_bear - 0.5 * (orb_low - L1_bear)

            for col, series in [
                (f"orb_high_{nm}", orb_high),
                (f"orb_low_{nm}",  orb_low),
                (f"orb_mid_{nm}",  mid),
                (f"L1_bull_{nm}",  L1_bull),
                (f"L2_bull_{nm}",  L2_bull),
                (f"L1_bear_{nm}",  L1_bear),
                (f"L2_bear_{nm}",  L2_bear),
            ]:
                out[col] = pd.NA
                out.loc[in_session, col] = series
        else:
            # create empty columns so schema is stable
            for col in [f"orb_high_{nm}", f"orb_low_{nm}", f"orb_mid_{nm}",
                        f"L1_bull_{nm}", f"L2_bull_{nm}", f"L1_bear_{nm}", f"L2_bear_{nm}"]:
                out[col] = pd.NA

    return out

# ---------- Candlestick stats

def candle_stats(df: pd.DataFrame) -> pd.DataFrame:
    o = df["open"]; h = df["high"]; l = df["low"]; c = df["close"]
    body = (c - o).abs()
    rng = (h - l).replace(0, np.nan)
    upper_wick = (h - np.maximum(c, o))
    lower_wick = (np.minimum(c, o) - l)
    stats = pd.DataFrame(index=df.index)
    stats["candle_color"] = np.where(c > o, "bull", np.where(c < o, "bear", "doji"))
    stats["body"] = body
    stats["range"] = (h - l)
    stats["upper_wick"] = upper_wick
    stats["lower_wick"] = lower_wick
    stats["body_pct_of_range"] = (body / rng).fillna(0.0)
    stats["upper_wick_pct_of_range"] = (upper_wick / rng).fillna(0.0)
    stats["lower_wick_pct_of_range"] = (lower_wick / rng).fillna(0.0)
    stats["is_doji"] = stats["body_pct_of_range"] < 0.1
    stats["is_marubozu"] = (stats["upper_wick_pct_of_range"] < 0.05) & (stats["lower_wick_pct_of_range"] < 0.05)
    return stats

# ---------- Binance via ccxt

class BinanceData:
    def __init__(self, rate_limit_ms: int = 1200):
        self.exchange = ccxt.binance({
            "enableRateLimit": True,
            "rateLimit": rate_limit_ms,
            "options": {"defaultType": "spot"},
        })
        self.exchange.load_markets()  # required

    def _raw_klines(self, symbol: str, interval: str, start_ms: int, limit: int = 1000) -> List[List[Any]]:
        market = self.exchange.market(symbol)  # 'BTC/USDT' -> {'id': 'BTCUSDT', ...}
        params = {"symbol": market["id"], "interval": interval, "limit": limit, "startTime": start_ms}
        return self.exchange.publicGetKlines(params)

    def fetch_ohlcv_batched(
        self, symbol: str, timeframe: str, since_ms: int, until_ms: Optional[int] = None,
        step_limit: int = 1000, sleep_ms: int = 0
    ) -> pd.DataFrame:
        tf = interval_to_ccxt_tf(timeframe)
        all_rows = []
        next_ms = int(since_ms)
        if until_ms is None:
            until_ms = to_millis(datetime.now(timezone.utc))
        else:
            until_ms = int(until_ms)

        while True:
            rows = self._raw_klines(symbol, tf, next_ms, limit=step_limit)
            if not rows:
                break
            all_rows.extend(rows)

            last_close = int(rows[-1][6])  # ensure int
            next_ms = last_close + 1

            if next_ms >= until_ms:
                break
            if sleep_ms > 0:
                time.sleep(sleep_ms / 1000.0)

        if not all_rows:
            return pd.DataFrame(columns=[
                "open","high","low","close","volume","quote_volume","n_trades","taker_buy_base_vol","taker_buy_quote_vol"
            ])

        cols = ["open_time","open","high","low","close","volume","close_time",
                "quote_volume","n_trades","taker_buy_base_vol","taker_buy_quote_vol","_ignore"]
        df = pd.DataFrame(all_rows, columns=cols)

        # Cast numerics (some envs return strings)
        num = ["open","high","low","close","volume","quote_volume","taker_buy_base_vol","taker_buy_quote_vol"]
        df[num] = df[num].astype(float)
        df["n_trades"] = df["n_trades"].astype(int)

        # Times → datetime
        df["open_time"] = pd.to_datetime(df["open_time"].astype("int64"), unit="ms", utc=True)
        df["close_time"] = pd.to_datetime(df["close_time"].astype("int64"), unit="ms", utc=True)

        df = df.set_index("open_time").sort_index()
        return df.drop(columns=["_ignore"])

    # ---- Top N by previous-day quote-volume gain
    def screen_top_by_prevday_volume_gain(
        self, usdt_only: bool = True, top_n: int = 10,
        min_price: float = 0.0, min_vol_quote: float = 0.0
    ) -> List[str]:
        markets = self.exchange.load_markets()
        symbols = [m for m in markets if markets[m]["active"]]
        if usdt_only:
            symbols = [s for s in symbols if s.endswith("/USDT")]

        tickers = self.exchange.fetch_tickers(symbols)
        filtered = []
        for s in symbols:
            t = tickers.get(s, {})
            last = t.get("last")
            quote_vol = t.get("quoteVolume")  # 24h quote vol
            if (last is None or (min_price and last < min_price) or
                (quote_vol is None or (min_vol_quote and quote_vol < min_vol_quote))):
                continue
            filtered.append(s)

        gains = []
        for s in filtered:
            try:
                daily = self.fetch_ohlcv_batched(
                    s, "1d",
                    since_ms=to_millis(datetime.now(timezone.utc) - timedelta(days=10))
                )
                if len(daily) < 3:
                    continue
                vol = daily["quote_volume"].dropna()
                gain = (vol.iloc[-2] - vol.iloc[-3]) / max(vol.iloc[-3], 1e-9)
                gains.append((s, gain))
            except Exception:
                continue

        gains.sort(key=lambda x: x[1], reverse=True)
        return [s for s, _ in gains[:top_n]]
    
    def screen_symbols_by_24h_usdt_volume(
        self,
        min_usd: float = 10_000_000.0,
        usdt_only: bool = True,
    ) -> list[str]:
        """
        Return */USDT symbols whose 24h quoteVolume >= min_usd.
        For */USDT, quoteVolume is already in USDT (≈ USD).
        """
        markets = self.exchange.load_markets()
        symbols = [s for s, m in markets.items() if m.get("active")]
        if usdt_only:
            symbols = [s for s in symbols if s.endswith("/USDT")]

        tickers = self.exchange.fetch_tickers(symbols)  # one call
        selected = [s for s in symbols
                    if tickers.get(s, {}).get("quoteVolume") is not None
                    and float(tickers[s]["quoteVolume"]) >= float(min_usd)]
        # sort descending by 24h quote volume for convenience
        selected.sort(key=lambda s: tickers.get(s, {}).get("quoteVolume", 0.0), reverse=True)
        return selected

# ---------- Pipeline

class CryptoDataPipeline:
        
    def __init__(self,
                 timeframe: str = "15m",
                 start_date: str = "2024-01-01",
                 symbols: Optional[List[str]] = None,
                 screen_top_prevday_gain: Optional[int] = None,
                 rate_limit_ms: int = 1200,
                 sessions: Optional[List[Dict[str, str]]] = None,
                 sort_mode: str = "symbol_then_time",
                 min_24h_usdt_volume: Optional[float] = None  # <-- NEW
                 ):
        """
        sessions: list of {"name","start_utc","end_utc"}; defaults to DEFAULT_SESSIONS
        sort_mode: "symbol_then_time" (MultiIndex) or "time" (interleaved)
        """
        self.timeframe = timeframe
        self.start_ms = to_millis(datetime.fromisoformat(start_date).replace(tzinfo=timezone.utc))
        self.binance = BinanceData(rate_limit_ms=rate_limit_ms)
        self.sessions = sessions if sessions is not None else DEFAULT_SESSIONS
        self.sort_mode = sort_mode
        
      
        if symbols is not None:
            self.symbols = ensure_usdt_symbols(symbols)
        elif min_24h_usdt_volume is not None:  # <-- NEW branch
            self.symbols = self.binance.screen_symbols_by_24h_usdt_volume(
                min_usd=min_24h_usdt_volume, usdt_only=True
            )
        elif screen_top_prevday_gain:
            self.symbols = self.binance.screen_top_by_prevday_volume_gain(
                usdt_only=True, top_n=screen_top_prevday_gain
            )
        else:
            raise ValueError("Provide symbols OR min_24h_usdt_volume OR screen_top_prevday_gain")

        if not self.symbols:
            raise ValueError("No symbols selected after filtering.")
        
        

    def build_dataframe_for_symbol(self, symbol: str) -> pd.DataFrame:
        raw = self.binance.fetch_ohlcv_batched(symbol, self.timeframe, since_ms=self.start_ms, step_limit=1000, sleep_ms=0)
        if raw.empty:
            return raw

        df = raw.copy()
        df = df[[
            "open","high","low","close","volume",
            "quote_volume","n_trades","taker_buy_base_vol","taker_buy_quote_vol","close_time"
        ]]

        # Volume in USDT (quote_volume already USDT for */USDT)
        df["volume_usdt"] = df["quote_volume"].where(df["quote_volume"] > 0, df["close"] * df["volume"])

        # Calendar + technicals
        df["dow"] = df.index.dayofweek
        for span in (9, 20, 100, 200):
            df[f"ema_{span}"] = ema(df["close"], span)
        macd_line, macd_signal, macd_hist = macd(df["close"])
        df["macd_line"] = macd_line; df["macd_signal"] = macd_signal; df["macd_hist"] = macd_hist
        df["rsi_14"] = rsi(df["close"], 14)

        # Daily ORB (UTC) and per-session ORBs
        df = mark_orb_daily(df)
        df = add_sessions_and_orbs(df, self.sessions)

        # Volume percentile bands
        p20, p50, p80 = rolling_percentiles(df["volume_usdt"], window=50, percentiles=(0.2, 0.5, 0.8))
        df["vol_p20"] = p20; df["vol_p50"] = p50; df["vol_p80"] = p80

        # Candle stats
        stats = candle_stats(df.rename(columns=str))
        df = pd.concat([df, stats], axis=1)

        # Flow proxy
        with np.errstate(divide='ignore', invalid='ignore'):
            df["taker_buy_share"] = (df["taker_buy_base_vol"] / df["volume"]).replace([np.inf, -np.inf], np.nan).fillna(0.0)
        df["bull_bear_vol_ratio"] = df["taker_buy_share"] / (1 - df["taker_buy_share"] + 1e-9)

        df["symbol"] = symbol

        preferred = [
            "symbol","open","high","low","close","volume","volume_usdt","quote_volume","n_trades",
            "taker_buy_base_vol","taker_buy_quote_vol","taker_buy_share","bull_bear_vol_ratio",
            "dow","ema_9","ema_20","ema_100","ema_200",
            "macd_line","macd_signal","macd_hist","rsi_14",
            # Daily ORB
            "is_orb","orb_high","orb_low","orb_mid","L1_bull","L2_bull","L1_bear","L2_bear",
            # (session columns will follow)
            "vol_p20","vol_p50","vol_p80",
            "candle_color","body","range","upper_wick","lower_wick",
            "body_pct_of_range","upper_wick_pct_of_range","lower_wick_pct_of_range",
            "is_doji","is_marubozu","close_time"
        ]
        cols = [c for c in preferred if c in df.columns] + [c for c in df.columns if c not in preferred]
        return df[cols]

    def run(self) -> pd.DataFrame:
        frames, syms = [], []
        for s in self.symbols:
            try:
                frames.append(self.build_dataframe_for_symbol(s))
                syms.append(s)
            except Exception as e:
                print(f"[WARN] {s} failed: {e}")
        if not frames:
            return pd.DataFrame()

        if self.sort_mode == "symbol_then_time":
            out = pd.concat(frames, keys=syms, names=["symbol", "time"])
            out["symbol"] = out.index.get_level_values("symbol")  # convenience
            return out
        else:
            out = pd.concat(frames).sort_index()
            return out


In [ ]:


if __name__ == "__main__":
    pipeline = CryptoDataPipeline(
        timeframe="15m",
        start_date="2025-01-01",
        min_24h_usdt_volume=10_000_000.0,   # <- NEW: only symbols ≥ $10m 24h quoteVolume
        sessions=DEFAULT_SESSIONS,
        sort_mode="symbol_then_time"
    )
    df = pipeline.run()
    df.to_csv("crypto_data.csv", index=True)

In [ ]:
print(f"{len(pipeline.symbols)} symbols selected:", pipeline.symbols[:20])

**PLOT**

In [ ]:
# bokeh_orb_multisession_symbol_select_autoscale_limited_alwaysWM.py
from __future__ import annotations
import os
from datetime import datetime
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_file
from bokeh.models import (
    ColumnDataSource, HoverTool, Label, CustomJS, CheckboxButtonGroup, Select,
    NumeralTickFormatter, DatetimeTickFormatter, Range1d, Span, BoxAnnotation
)
from bokeh.layouts import column, row

# ----------------------------
# CONFIG
# ----------------------------
CSV_PATH    = r"c:\Users\aadegbola\Projects\ORB\crypto_data.csv"
OUTPUT_DIR  = r"c:\Users\aadegbola\Projects\ORB"
SESSIONS    = ["asia", "europe", "us", "overnight"]
SESSION_LABEL = { "asia":"ASIA", "europe":"EUROPE", "us":"US", "overnight":"OVERNIGHT" }
TIMEFRAME   = "timeframe" #"15m"
TITLE_PREFIX = "ORB Multi-Session Plot"

def _sess_range_text(t0, t1):
    return f"{t0.strftime('%H:%M')}–{t1.strftime('%H:%M')}"

# ----------------------------
# LOAD
# ----------------------------
df = pd.read_csv(CSV_PATH, low_memory=False)
df["time"] = pd.to_datetime(df["time"], errors="coerce", utc=True)

symbols = sorted(df["symbol"].dropna().unique().tolist())
if not symbols:
    raise ValueError("No symbols found in CSV")

NUM_SESS = len(SESSIONS)

# ----------------------------
# ACTIVE SOURCES (swap on symbol change)
# ----------------------------
active_price = ColumnDataSource(dict(
    time_i=[], open=[], high=[], low=[], close=[], body_top=[], body_bot=[], body_color=[]
))
active_vol = ColumnDataSource(dict(time_i=[], vol=[], c=[]))

price_sources, vol_sources = {}, {}

# Overlay containers per symbol/session:
#  - static_overlays: vertical session lines + watermark labels (ALWAYS visible for selected symbol)
#  - toggle_overlays: ORB candle band, ORB band, level lines (solid+dotted) + level labels (toggle with checkbox)
static_overlays: list[list[list]] = []   # [sym][sess] -> list of models
toggle_overlays: list[list[list]] = []   # [sym][sess] -> list of models

symbol_title_text = []
price_min, price_max, vol_min, vol_max = [], [], [], []

# ----------------------------
# FIGURES (no grid lines)
# ----------------------------
p_price = figure(x_axis_type="datetime", width=1200, height=520, toolbar_location="right")
p_price.title.text = f"{TITLE_PREFIX}: — ({TIMEFRAME})"
p_price.xaxis.formatter = DatetimeTickFormatter(hours="%H:%M")
p_price.yaxis.formatter = NumeralTickFormatter(format="0,0")
p_price.xgrid.grid_line_color = None
p_price.ygrid.grid_line_color = None

p_vol = figure(x_axis_type="datetime", width=1200, height=240, toolbar_location=None, x_range=p_price.x_range)
p_vol.yaxis.axis_label = "Volume (USDT)"
p_vol.yaxis.formatter = NumeralTickFormatter(format="0,0")
p_vol.xgrid.grid_line_color = None
p_vol.ygrid.grid_line_color = None

# Price glyphs (bound to ACTIVE sources)
wicks = p_price.segment("time_i", "high", "time_i", "low", color="black", line_width=1, source=active_price)
bodies = p_price.vbar(x="time_i", top="body_top", bottom="body_bot", width=1,
                      fill_color="body_color", line_color="black", source=active_price)
p_price.add_tools(HoverTool(
    renderers=[bodies],
    tooltips=[("Time", "@time_i{%F %T}"),
              ("Open", "@open{0,0.00}"),
              ("Close","@close{0,0.00}"),
              ("High", "@high{0,0.00}"),
              ("Low",  "@low{0,0.00}")],
    formatters={"@time_i": "datetime"},
    mode="vline"
))

p_vol_bar = p_vol.vbar(x="time_i", top="vol", width=1, fill_color="c", line_color=None, source=active_vol)
p_vol.add_tools(HoverTool(
    tooltips=[("Time", "@time_i{%F %T}"), ("Vol USDT","@vol{0,0}")],
    formatters={"@time_i": "datetime"},
    mode="vline"
))

# ----------------------------
# BUILD PER-SYMBOL STRUCTURES
# ----------------------------
def build_symbol(symbol: str, sym_idx: int):
    d = df[df["symbol"] == symbol].copy()
    if d.empty:
        return

    # Robust anchor day selection: prefer latest US session, else any session with data, else last available date
    candidate_cols = ["session_id_us"] + [f"session_id_{s}" for s in SESSIONS]
    anchor_token = None
    for col in candidate_cols:
        if col in d.columns and d[col].dropna().shape[0] > 0:
            anchor_token = d[col].dropna().iloc[-1]
            break
    if anchor_token is not None:
        date_str = str(anchor_token).split("_")[0]
        day0 = pd.Timestamp(date_str, tz="UTC")
    else:
        last_t = pd.to_datetime(d["time"], utc=True).max()
        day0 = last_t.normalize()
    day1 = day0 + pd.Timedelta(days=1)

    dd = d[(d["time"] >= day0) & (d["time"] < day1)].copy().sort_values("time")
    if dd.empty:
        return

    symbol_title_text.append(f"{TITLE_PREFIX}: {symbol} — {day0.date()}  ({TIMEFRAME})")

    # bar width
    tvals = dd["time"].values
    step_ms = int(np.median(np.diff(tvals).astype("timedelta64[ms]").astype(np.int64))) if len(tvals) > 1 else 60_000
    bar_width_ms = int(step_ms * 0.8)

    # price/volume fields
    dd["time_i"] = dd["time"]
    dd["body_top"] = np.maximum(dd["open"].values, dd["close"].values)
    dd["body_bot"] = np.minimum(dd["open"].values, dd["close"].values)
    dd["body_color"] = np.where(dd["close"] >= dd["open"], "#2ca02c", "#d62728")

    p20 = dd["vol_p20"].ffill().bfill() if "vol_p20" in dd else pd.Series(np.nan, index=dd.index)
    p50 = dd["vol_p50"].ffill().bfill() if "vol_p50" in dd else pd.Series(np.nan, index=dd.index)
    p80 = dd["vol_p80"].ffill().bfill() if "vol_p80" in dd else pd.Series(np.nan, index=dd.index)
    def vcol(v,a,b,c):
        if np.isnan(a) or np.isnan(b) or np.isnan(c): return "#b0b0b0"
        if v < a:  return "#a8dadc"
        if v < b:  return "#457b9d"
        if v < c:  return "#1d3557"
        return "#e76f51"
    dd["c"] = [vcol(v,a,b,c) for v,a,b,c in zip(dd["volume_usdt"], p20, p50, p80)]

    price_sources[symbol] = ColumnDataSource(dd[["time_i","open","high","low","close","body_top","body_bot","body_color"]])
    vol_sources[symbol]   = ColumnDataSource(dd[["time_i","volume_usdt","c"]].rename(columns={"volume_usdt":"vol"}))

    # y-ranges (with padding)
    pmin = float(dd["low"].min()); pmax = float(dd["high"].max())
    pad = 0.01 * max(1e-9, (pmax - pmin))
    price_min.append(pmin - pad); price_max.append(pmax + pad)

    vmax = float(dd["volume_usdt"].max()) if "volume_usdt" in dd else 1.0
    vol_min.append(0.0); vol_max.append(max(1.0, vmax * 1.05))

    # ----------------------------
    # overlays per session
    # ----------------------------
    stat_per_session = []
    togg_per_session = []

    y_top = float(dd["high"].max())
    x_left, x_right = dd["time"].min(), dd["time"].max()

    for s in SESSIONS:
        stat_items, togg_items = [], []
        sid_col = f"session_id_{s}"
        is_orb_col = f"is_orb_{s}"
        needed = [sid_col, is_orb_col,
                  f"orb_high_{s}", f"orb_low_{s}", f"orb_mid_{s}",
                  f"L1_bull_{s}", f"L2_bull_{s}", f"L1_bear_{s}", f"L2_bear_{s}"]
        if not all(c in dd.columns for c in needed) or dd[sid_col].dropna().empty:
            # keep alignment even if session columns are missing for this symbol/day
            stat_per_session.append(stat_items); togg_per_session.append(togg_items); continue

        rows = dd.dropna(subset=[sid_col]).copy()
        t0, t1 = rows["time"].min(), rows["time"].max()

        # STATIC: vertical boundaries (always visible for selected symbol)
        vstart = Span(location=t0, dimension="height", line_color="#bdbdbd",
                      line_dash="dashed", line_width=1, visible=False)
        vend   = Span(location=t1, dimension="height", line_color="#bdbdbd",
                      line_dash="dashed", line_width=1, visible=False)
        p_price.add_layout(vstart); p_price.add_layout(vend)
        stat_items += [vstart, vend]

        # STATIC: watermark
        wm = Label(x=t0 + (t1 - t0)/2, y=y_top, y_offset=-8,
                   text=f"{SESSION_LABEL[s]}  {_sess_range_text(pd.to_datetime(t0).to_pydatetime(), pd.to_datetime(t1).to_pydatetime())}",
                   text_color="#9e9e9e", text_font_size="12pt", text_alpha=0.35,
                   text_font_style="bold", visible=False)
        p_price.add_layout(wm); stat_items.append(wm)

        # TOGGLE: ORB candle highlight
        ob = rows.loc[rows[is_orb_col].astype(bool)].iloc[0] if rows[is_orb_col].any() else rows.iloc[0]
        band = BoxAnnotation(
            left=ob["time"] - pd.Timedelta(milliseconds=bar_width_ms/2),
            right=ob["time"] + pd.Timedelta(milliseconds=bar_width_ms/2),
            bottom=ob["low"], top=ob["high"],
            fill_alpha=0.35, fill_color="#ffd166", line_alpha=0, visible=False
        )
        p_price.add_layout(band); togg_items.append(band)

        # TOGGLE: ORB levels (solid inside session; light dotted outside)
        orb_high = float(rows[f"orb_high_{s}"].iloc[0])
        orb_low  = float(rows[f"orb_low_{s}"].iloc[0])
        orb_mid  = float(rows[f"orb_mid_{s}"].iloc[0])
        L1b = float(rows[f"L1_bull_{s}"].iloc[0]); L2b = float(rows[f"L2_bull_{s}"].iloc[0])
        L1s = float(rows[f"L1_bear_{s}"].iloc[0]); L2s = float(rows[f"L2_bear_{s}"].iloc[0])

        col_orb = "#64b5f6"; col_mid = "#ba68c8"
        col_bull1, col_bull2 = "#81c784", "#66bb6a"
        col_bear1, col_bear2 = "#e57373", "#ef5350"

        def draw_level(y, color, text):
            ln_in = p_price.line(x=[t0, t1], y=[y, y], color=color, line_width=1, line_alpha=1.0, visible=False)
            if t0 > x_left:
                ln_l = p_price.line(x=[x_left, t0], y=[y, y], color=color, line_dash="dotted",
                                    line_width=1, line_alpha=0.45, visible=False); togg_items.append(ln_l)
            if t1 < x_right:
                ln_r = p_price.line(x=[t1, x_right], y=[y, y], color=color, line_dash="dotted",
                                    line_width=1, line_alpha=0.45, visible=False); togg_items.append(ln_r)
            lab = Label(x=x_right, y=y, x_offset=6, text=text,
                        text_color=color, text_font_size="8pt", text_alpha=0.85, visible=False)
            p_price.add_layout(lab)
            togg_items.extend([ln_in, lab])

        draw_level(orb_high, col_orb, f"{SESSION_LABEL[s]} H")
        draw_level(orb_low,  col_orb, f"{SESSION_LABEL[s]} L")
        draw_level(orb_mid,  col_mid, f"{SESSION_LABEL[s]} M")
        draw_level(L1b,      col_bull1, f"{SESSION_LABEL[s]} L1↑")
        draw_level(L2b,      col_bull2, f"{SESSION_LABEL[s]} L2↑")
        draw_level(L1s,      col_bear1, f"{SESSION_LABEL[s]} L1↓")
        draw_level(L2s,      col_bear2, f"{SESSION_LABEL[s]} L2↓")

        # TOGGLE: ORB price band shading
        band_price = BoxAnnotation(left=t0, right=t1, bottom=orb_low, top=orb_high,
                                   fill_alpha=0.10, fill_color="#2196F3", line_alpha=0, visible=False)
        p_price.add_layout(band_price); togg_items.append(band_price)

        stat_per_session.append(stat_items)
        togg_per_session.append(togg_items)

    static_overlays.append(stat_per_session)
    toggle_overlays.append(togg_per_session)

# Build all symbols
for i, sym in enumerate(symbols):
    build_symbol(sym, i)

# ----------------------------
# UI
# ----------------------------
symbol_select = Select(title="Symbol", value=symbols[0], options=symbols)
cbox = CheckboxButtonGroup(labels=[SESSION_LABEL[s] for s in SESSIONS],
                           active=list(range(NUM_SESS)))  # default: show all sessions' ORB lines

# Seed initial data
def _copy_src(dst: ColumnDataSource, src: ColumnDataSource):
    dst.data = {k: np.array(v) for k, v in src.data.items()}

_copy_src(active_price, price_sources[symbols[0]])
_copy_src(active_vol,   vol_sources[symbols[0]])

# Initial bar width
def _bar_width_from_src(src: ColumnDataSource) -> int:
    t = pd.to_datetime(src.data["time_i"])
    if len(t) <= 1: return 60000
    step = int(np.median(np.diff(t.view("int64")) // 1_000_000))  # ns->ms
    return int(step * 0.8)

default_width = _bar_width_from_src(price_sources[symbols[0]])
bodies.glyph.width = default_width
p_vol_bar.glyph.width = default_width

# Initial ranges & title
pmin0 = float(price_min[0]); pmax0 = float(price_max[0])
vmin0 = float(vol_min[0]);   vmax0 = float(vol_max[0])
p_price.y_range = Range1d(pmin0, pmax0); p_vol.y_range = Range1d(vmin0, vmax0)
p_price.title.text = symbol_title_text[0]

# Show default symbol’s STATIC overlays (always on for selected symbol)
for objs in static_overlays[0]:
    for obj in objs:
        obj.visible = True

# Show default symbol’s TOGGLE overlays (respect checkbox state)
for s_idx, objs in enumerate(toggle_overlays[0]):
    vis = s_idx in cbox.active
    for obj in objs:
        obj.visible = vis

# ----------------------------
# JS CALLBACKS
# ----------------------------
js_args = {
    "active_price": active_price,
    "active_vol": active_vol,
    "bodies": bodies.glyph,
    "volbars": p_vol_bar.glyph,
    "symbols": symbols,
    "title_model": p_price.title,
    "rng_price": p_price.y_range,
    "rng_vol": p_vol.y_range,
    "cbox": cbox,
    "num_sess": NUM_SESS,
}

# pass per-symbol sources/ranges/titles
for i, sym in enumerate(symbols):
    js_args[f"src_p_{i}"] = price_sources[sym]
    js_args[f"src_v_{i}"] = vol_sources[sym]
    js_args[f"title_txt_{i}"] = symbol_title_text[i]
    js_args[f"rng_price_min_{i}"] = price_min[i]
    js_args[f"rng_price_max_{i}"] = price_max[i]
    js_args[f"rng_vol_min_{i}"]   = vol_min[i]
    js_args[f"rng_vol_max_{i}"]   = vol_max[i]
    # static & toggle overlays
    for s_idx, obj_list in enumerate(static_overlays[i]):
        for j, obj in enumerate(obj_list):
            js_args[f"stat_{i}_{s_idx}_{j}"] = obj
    for s_idx, obj_list in enumerate(toggle_overlays[i]):
        for j, obj in enumerate(obj_list):
            js_args[f"togg_{i}_{s_idx}_{j}"] = obj

# Symbol change:
#  - swap data, recompute bar width
#  - title & autoscale ranges
#  - HIDE all static & toggle overlays for all symbols
#  - SHOW static overlays for the selected symbol (ALL sessions)
#  - SHOW toggle overlays for the selected symbol according to checkboxes
sym_change_code = """
const sym = cb_obj.value;
const idx = symbols.indexOf(sym);

// swap data
active_price.data = {...eval(`src_p_${idx}`).data};
active_vol.data   = {...eval(`src_v_${idx}`).data};
active_price.change.emit(); active_vol.change.emit();

// widths
const t = active_price.data['time_i'];
let width = 60000;
if (t && t.length > 1) {
  const ms = t.map(x => (new Date(x)).getTime());
  let diffs = [];
  for (let k=1;k<ms.length;k++) diffs.push(ms[k]-ms[k-1]);
  diffs.sort((a,b)=>a-b);
  const med = diffs[Math.floor(diffs.length/2)];
  width = Math.floor(med * 0.8);
}
bodies.width = width; volbars.width = width;

// title & ranges
title_model.text = `${eval(`title_txt_${idx}`)}`;
rng_price.start = eval(`rng_price_min_${idx}`);
rng_price.end   = eval(`rng_price_max_${idx}`);
rng_vol.start   = eval(`rng_vol_min_${idx}`);
rng_vol.end     = eval(`rng_vol_max_${idx}`);

// hide ALL static & toggle overlays
for (let i=0; i<symbols.length; i++){
  for (let s=0; s<num_sess; s++){
    for (let j=0;; j++){ try { eval(`stat_${i}_${s}_${j}`).visible = false; } catch(e){ break; } }
    for (let j=0;; j++){ try { eval(`togg_${i}_${s}_${j}`).visible = false; } catch(e){ break; } }
  }
}

// show STATIC overlays for selected symbol (always visible)
for (let s=0; s<num_sess; s++){
  for (let j=0;; j++){
    try { eval(`stat_${idx}_${s}_${j}`).visible = true; }
    catch(e){ break; }
  }
}

// show TOGGLE overlays according to checkboxes
const act = new Set(cbox.active);
for (let s=0; s<num_sess; s++){
  const show = act.has(s);
  for (let j=0;; j++){
    try { eval(`togg_${idx}_${s}_${j}`).visible = show; }
    catch(e){ break; }
  }
}
"""
symbol_select.js_on_change("value", CustomJS(args=js_args, code=sym_change_code))

# Checkbox change: affects only TOGGLE overlays of current symbol
js_args_cb = dict(js_args)
js_args_cb["symbol_select"] = symbol_select
cbox_callback_code = """
const idx = symbols.indexOf(symbol_select.value);
const act = new Set(cb_obj.active);
for (let s=0; s<num_sess; s++){
  const show = act.has(s);
  for (let j=0;; j++){
    try { eval(`togg_${idx}_${s}_${j}`).visible = show; }
    catch(e){ break; }
  }
}
"""
cbox.js_on_change("active", CustomJS(args=js_args_cb, code=cbox_callback_code))

# ----------------------------
# SAVE
# ----------------------------
os.makedirs(OUTPUT_DIR, exist_ok=True)
stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
file_name = f"orb_plot_multi_symbols_autoscale_limited_alwaysWM_{TIMEFRAME}_{stamp}.html"
full_path = os.path.join(OUTPUT_DIR, file_name)

output_file(full_path, title=f"{TITLE_PREFIX} — ({TIMEFRAME})")
show(column(row(symbol_select, cbox), p_price, p_vol))
print(f"✅ Saved: {full_path}")


✅ Saved: c:\Users\aadegbola\Projects\ORB\orb_plot_multi_symbols_autoscale_limited_alwaysWM_timeframe_20250926_002735.html
